# Rakuten Travel レビュー収集スクリプト（2024年9月）

このスクリプトは、楽天トラベルに掲載されている全国のホテルレビューのうち、**2024年9月に投稿されたレビュー**を都道府県ごとに**最大23件ずつ**収集するPythonコードです。スクレイピングの実行には数分かかります。

各県の収集する件数を増やすときは適宜該当箇所を編集して実行してください。
---

## 特徴

- **対象期間**：2024年9月に宿泊したレビュー
- **対象地域**：全国47都道府県（各県最大23件）
- **対象ページ**：`https://travel.rakuten.co.jp/yado/<slug>/` を中心に複数のリストパターンからホテル一覧を取得
- **並列処理**：`ThreadPoolExecutor(max_workers=8)` により、ホテル詳細ページの取得を高速化
- **進捗表示**：標準出力にて進捗状況を表示、レビュー数が不足している県は⚠マーク付きで通知
- **保存形式**：`japan_202409_reviews.json` に結果を保存（UTF-8）


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Rakuten Travel レビュー収集 2024‑09（全国・県別 23 件上限・進捗表示）
──────────────────────────────────────────────
◎ 各都道府県 23 件：本文あり / 宿泊年月 2024‑09
◎ /yado/<slug>/ をメイン一覧として使用
◎ ThreadPoolExecutor(max_workers=8) で詳細ページを並列取得
◎ 県ごとに不足があれば ⚠ を表示し、終了時にサマリ
"""

import re, json, time, random
from pathlib import Path
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

import requests
from bs4 import BeautifulSoup

# ─────────── 設定 ───────────
MAX_PER_PREF = 23
TARGET_MONTH = "2024年09月"
MAX_WORKERS  = 8
SLEEP_RANGE  = (0.2, 0.5)
OUT_FILE     = "japan_202409_reviews.json"
UA           = {"User-Agent": "Mozilla/5.0"}

PREF_SLUGS = [
    "hokkaido","aomori","iwate","miyagi","akita","yamagata","fukushima",
    "ibaraki","tochigi","gunma","saitama","chiba","tokyo","kanagawa",
    "niigata","toyama","ishikawa","fukui","yamanashi","nagano",
    "gifu","shizuoka","aichi","mie",
    "shiga","kyoto","osaka","hyogo","nara","wakayama",
    "tottori","shimane","okayama","hiroshima","yamaguchi",
    "tokushima","kagawa","ehime","kochi",
    "fukuoka","saga","nagasaki","kumamoto","oita","miyazaki",
    "kagoshima","okinawa"
]

# ─────────── HTTP with retry ───────────
_session = requests.Session()
def http_get(url: str, **kw):
    for back in (1, 2, 4):
        try:
            r = _session.get(url, headers=UA, timeout=10, **kw)
            if r.status_code == 200:
                return r
        except requests.RequestException:
            pass
        time.sleep(back)
    return None

# ─────────── hotel list ───────────
PAT_ID = re.compile(r"/(?:HOTEL|hotelinfo)/(\d+)/", re.I)

LIST_PATTERNS = [
    # ① 現行 /yado/<slug>/ （ページ 2 以降は ?page=2）
    "https://travel.rakuten.co.jp/yado/{slug}/?page={page}",
    # ② グループ別旧 URL
    "https://travel.rakuten.co.jp/group/tiku/02japan{slug}.html?p={page}",
    # ③ 事務所別旧 URL
    "https://travel.rakuten.co.jp/office/pref/{slug}.html?p={page}",
    # ④ 観光ページ
    "https://travel.rakuten.co.jp/group/kanko/02japan{slug}.html?p={page}",
    # ⑤ 温泉ページ
    "https://travel.rakuten.co.jp/group/onsen/02japan{slug}.html?p={page}",
]

def fetch_hotel_ids(slug: str, max_pages: int = 50):
    ids = set()
    for base in LIST_PATTERNS:
        for p in range(1, max_pages + 1):
            url = base.format(slug=slug, page=p)
            res = http_get(url)
            if not res:
                break
            ids.update(PAT_ID.findall(res.text))
            # 「次へ」リンクが無ければ打ち切り
            if "次へ" not in res.text and "?page=" not in res.text:
                break
            time.sleep(random.uniform(*SLEEP_RANGE))
        if ids:          # どれか 1 パターンで取れれば十分
            break
    return list(ids)

# ─────────── 詳細ページ → レビュー 1 件 ───────────
LINK_SEL = (
    "p.reviewTitle a, h2.commentTitle a, "
    "div.rvTtl a, div.rvTltl a"
)

def fetch_review(hid: str, slug: str):
    lst = http_get(
        f"https://review.travel.rakuten.co.jp/hotel/voice/{hid}/",
        params={"f_time":"202409","f_sort":"0","f_next":"0"}
    )
    if not lst:
        return None, "一覧取得失敗"

    soup = BeautifulSoup(lst.text, "html.parser")
    a = soup.select_one(LINK_SEL)
    if not a:
        return None, "詳細リンク無し"

    det_url = a["href"]
    if det_url.startswith("/"):
        det_url = "https://review.travel.rakuten.co.jp" + det_url
    det = http_get(det_url)
    if not det:
        return None, "詳細取得失敗"

    sd = BeautifulSoup(det.text, "html.parser")
    body = sd.select_one("p.commentSentence")
    if not body or not body.get_text(strip=True):
        return None, "本文空"

    dds = sd.select("dl.commentPurpose dd")
    stay = dds[3].get_text(strip=True) if len(dds) >= 4 else ""
    if TARGET_MONTH not in stay:
        return None, "9月以外"

    return {
        "hotel_id": hid,
        "prefecture": slug,
        "stay_date": stay,
        "review_text": body.get_text(strip=True)
    }, None

# ─────────── メイン ───────────
def main():
    start = time.time()
    results, used_hotels = [], set()
    err_counter = defaultdict(int)

    with ThreadPoolExecutor(MAX_WORKERS) as pool:
        for slug in PREF_SLUGS:
            print(f"🌏 {slug}")
            hotel_ids = fetch_hotel_ids(slug)
            if not hotel_ids:
                print("  ⚠ hotel_ids 0 件")
                err_counter[(slug, "hotel_ids 0 件")] += 1
                continue
            random.shuffle(hotel_ids)

            futs = []
            for hid in hotel_ids:
                if len([r for r in results if r['prefecture'] == slug]) >= MAX_PER_PREF:
                    break
                if hid in used_hotels:
                    continue
                futs.append(pool.submit(fetch_review, hid, slug))
                used_hotels.add(hid)

            success = 0
            for fut in as_completed(futs):
                rev, err = fut.result()
                if rev:
                    results.append(rev)
                    success += 1
                    print(f"  ✔ {rev['hotel_id']} {rev['stay_date']}")
                else:
                    err_counter[(slug, err)] += 1

                if success >= MAX_PER_PREF:
                    break

            if success < MAX_PER_PREF:
                print(f"  ⚠ {MAX_PER_PREF - success} 件不足 ({success}/{MAX_PER_PREF})")

    # 保存
    Path(OUT_FILE).write_text(
        json.dumps(results, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    # サマリ
    print("\n━━ サマリ ━━━━━━━━━━━━━━━━━━━━━")
    goal = MAX_PER_PREF * len(PREF_SLUGS)
    print(f"期待: {goal} 件 | 実際: {len(results)} 件")
    if err_counter:
        print("不足理由 TOP5")
        for (slug, reason), cnt in sorted(err_counter.items(), key=lambda x: -x[1])[:5]:
            print(f"  {reason} [{slug}] … {cnt} 件")
    print(f"保存先: {OUT_FILE} (経過 {(time.time() - start)/60:.1f} 分)")

if __name__ == "__main__":
    main()
